In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [13]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2_block5_15AB_1FC_3.h5' #แก้
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x])

In [6]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/'#แก้
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')#แก้
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')#แก้
base_dir0 = '/media/tohn/SSD/Images/Image1/'#แก้
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [7]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Testdf.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1312, 22)
(1312, 22)
Normal:  (857, 22)
Abnormal:  (455, 22)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,filename
0,111,111,40,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,86.0,133.0,477.0,311.0,0.092664,0.148873,0.513514,0.346614,AB01 P1 C040.JPG
1,112,112,40,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,163.0,139.0,532.0,385.0,0.171698,0.154849,0.560377,0.428287,AB01 P2 C040.JPG
2,113,113,40,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,127.0,135.0,605.0,640.0,0.140316,0.150865,0.667984,0.711155,AB01 P4-1 C040.JPG
3,114,114,40,P5,P51,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,...,https://irisprodseatraining.blob.core.windows....,59.0,96.0,643.0,587.0,0.063462,0.107041,0.690385,0.653386,AB01 P5-1 C040.JPG
4,115,115,40,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,199.0,132.0,618.0,562.0,0.216374,0.146881,0.672515,0.625498,AB01 P3-1 C040.JPG


In [22]:
# dataframe.info()

In [8]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1312 validated image filenames belonging to 15 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}


In [14]:

from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [15]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.10783421,  0.31215259,  0.01733672, ...,  0.40046343,
         0.14020412,  0.37497926],
       [ 0.41059634,  0.66091961, -0.13656458, ...,  0.23649634,
         0.6331774 ,  0.09244446],
       [-0.01750612,  0.34510812,  0.22585644, ...,  0.24184038,
         0.3900131 , -0.1302004 ],
       ...,
       [ 0.3114734 ,  0.80987644, -0.18276721, ...,  1.19653237,
         0.14288557,  0.75333327],
       [ 0.2733095 , -0.02278921,  0.02211132, ...,  0.21971007,
        -0.15190458,  0.27240616],
       [ 0.18239088,  0.16288966, -0.16271284, ...,  0.35533658,
        -0.00221964,  0.65974861]])

In [16]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.107834,0.312153,0.017337,-0.183516,0.575318,-0.173991,-0.089335,-0.063324,-0.030139,0.059906,...,-0.192440,-0.159383,0.653832,-0.121368,0.138473,-0.108649,0.148072,0.400463,0.140204,0.374979
1,0.410596,0.660920,-0.136565,-0.185140,0.411923,-0.167171,0.004935,0.000345,-0.130163,-0.110029,...,-0.145701,-0.168994,0.213561,-0.044707,0.396031,-0.173826,-0.012626,0.236496,0.633177,0.092444
2,-0.017506,0.345108,0.225856,-0.165460,0.779619,-0.193235,-0.032824,-0.147713,-0.096544,0.065199,...,-0.125955,-0.136239,0.735197,-0.071152,0.282353,-0.138690,-0.005355,0.241840,0.390013,-0.130200
3,-0.020903,0.117675,-0.082832,-0.135977,0.282651,-0.220009,0.111934,-0.094878,-0.103359,0.036062,...,-0.179393,-0.162492,0.375171,-0.082910,0.298243,-0.055977,0.022169,0.057336,0.233832,-0.014271
4,0.064263,0.339076,0.294470,-0.157962,0.194468,-0.202357,-0.088951,-0.190239,-0.139355,-0.224386,...,-0.098921,-0.025334,0.175942,-0.027772,0.215621,0.174917,-0.076772,0.139161,0.083017,-0.109640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,0.744560,1.183852,-0.053521,-0.201383,0.486395,-0.161736,0.369923,0.262434,0.401869,0.046597,...,0.556529,-0.044270,0.239365,-0.006773,-0.007029,-0.143082,0.694543,0.309268,0.582698,0.681161
1308,0.002655,-0.022163,-0.106891,-0.050454,0.119425,0.226082,-0.119361,0.402494,0.157569,0.171320,...,0.420777,-0.108083,0.338136,0.070917,-0.033625,0.049808,0.067248,0.423208,-0.147667,0.224347
1309,0.311473,0.809876,-0.182767,-0.104543,0.685961,0.047535,0.061281,0.351369,0.239477,0.172855,...,-0.129816,-0.179644,0.494607,0.241752,0.758374,0.121146,0.329734,1.196532,0.142886,0.753333
1310,0.273309,-0.022789,0.022111,-0.126591,-0.079215,-0.175265,0.018625,-0.036004,0.046267,-0.022752,...,-0.036538,-0.101430,-0.133908,0.142786,0.102547,-0.169974,0.069229,0.219710,-0.151905,0.272406


In [17]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [18]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.107834,0.312153,0.017337,-0.183516,0.575318,-0.173991,-0.089335,-0.063324,-0.030139,0.059906,...,0.138473,-0.108649,0.148072,0.400463,0.140204,0.374979,Abnormal,P1,FP-A,AB01
1,0.410596,0.660920,-0.136565,-0.185140,0.411923,-0.167171,0.004935,0.000345,-0.130163,-0.110029,...,0.396031,-0.173826,-0.012626,0.236496,0.633177,0.092444,Abnormal,P2,FP-A,AB01
2,-0.017506,0.345108,0.225856,-0.165460,0.779619,-0.193235,-0.032824,-0.147713,-0.096544,0.065199,...,0.282353,-0.138690,-0.005355,0.241840,0.390013,-0.130200,Abnormal,P41,FP-B,AB01
3,-0.020903,0.117675,-0.082832,-0.135977,0.282651,-0.220009,0.111934,-0.094878,-0.103359,0.036062,...,0.298243,-0.055977,0.022169,0.057336,0.233832,-0.014271,Abnormal,P51,FP-C,AB01
4,0.064263,0.339076,0.294470,-0.157962,0.194468,-0.202357,-0.088951,-0.190239,-0.139355,-0.224386,...,0.215621,0.174917,-0.076772,0.139161,0.083017,-0.109640,Abnormal,P31,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,0.744560,1.183852,-0.053521,-0.201383,0.486395,-0.161736,0.369923,0.262434,0.401869,0.046597,...,-0.007029,-0.143082,0.694543,0.309268,0.582698,0.681161,Normal,P32,FP-B,Normal
1308,0.002655,-0.022163,-0.106891,-0.050454,0.119425,0.226082,-0.119361,0.402494,0.157569,0.171320,...,-0.033625,0.049808,0.067248,0.423208,-0.147667,0.224347,Normal,P42,FP-B,Normal
1309,0.311473,0.809876,-0.182767,-0.104543,0.685961,0.047535,0.061281,0.351369,0.239477,0.172855,...,0.758374,0.121146,0.329734,1.196532,0.142886,0.753333,Normal,P52,FP-C,Normal
1310,0.273309,-0.022789,0.022111,-0.126591,-0.079215,-0.175265,0.018625,-0.036004,0.046267,-0.022752,...,0.102547,-0.169974,0.069229,0.219710,-0.151905,0.272406,Normal,P61,FP-C,Normal


In [19]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FeaturesVector_Testset_15AB.csv')

In [15]:
# os.chdir( '/home/yupaporn/codes/USAI/')
# validationdf = dataset
# validationdf.to_csv('FeaturesVector_R2F1_B7.csv')

In [16]:
# os.chdir( '/home/yupaporn/codes/USAI/')
# testdf = dataset
# testdf.to_csv('FeaturesVector_valR2_Sub3.csv')